# PYTHON IMPLEMENTATION FOR PARAMETER VALUES OF THE CROP YIELD MODEL AFTER INCORPORATING TEMPERATURE

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math
np.random.seed(12)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## THE PREPARATION AND IMPUTATION OF MY TEMPERATURE AND YIELD DATA 

In [2]:
#Temperature data
tt=pd.read_excel('my_research_temp_data.xlsx')
data_in = np.array(tt['Annual Mean'])
T=data_temp = data_in[-62:]
years = np.array(tt['Category'])

#Crop Yield data
pp=pd.read_csv('crop yield.csv')
data_crop = np.array(pp['Value'])
years = np.array(pp['Year'])

# Log transformation
def log_transform(data_crop):
    # Adding 1 to avoid log(0) if there are zero values
    data = np.log(data_crop + 1)
    return data
data = log_transform(data_crop)

## ESTIMATION OF THE PARMETERS FOR THE CROP YIELD MODEL

In [3]:
# Step 1: Computation of Y and X
def compute_Y_X(data, T, delta_t, K):
    n = len(data)
    if n <= 1:
        raise ValueError("Data length must be greater than 1")

    Y = np.zeros(n - 1)
    X = np.zeros((n - 1, 2))
    
    for i in range(n - 1):
        sqrt_T_i = np.sqrt(T[i])
        Y[i] = (data[i + 1] - data[i]) / sqrt_T_i
        X[i, 0] = (1 - np.exp(data[i]) / K) * (delta_t / sqrt_T_i)  
        X[i, 1] = -(delta_t / sqrt_T_i)                            
    
    return Y, X

# Step 2: Computation of C_hat
def compute_C_hat(Y, X, lambda_reg=1e-5):
    n, p = X.shape
    I = np.eye(p)
    X_transpose_X_reg = X.T @ X + lambda_reg * I
    X_transpose_X_inv = np.linalg.inv(X_transpose_X_reg)
    C_hat = X_transpose_X_inv @ X.T @ Y
    return C_hat

# Step 3: Computation of Parameters
def compute_parameters(data, T, delta_t, K, lambda_reg=1e-5):
    Y, X = compute_Y_X(data, T, delta_t, K)
    C_hat = compute_C_hat(Y, X, lambda_reg)
    
    r = C_hat[0]
    sigma = -C_hat[1]
    
    # Predicted Model Y
    Y_pred = X @ C_hat
    residuals = Y - Y_pred
    
    # Calculation of Mean Squared Error (MSE)
    summ_sq_r = np.sum(residuals ** 2)
    total_summ_sq = np.sum((Y - np.mean(Y)) ** 2)
    mse = summ_sq_r / len(Y)
    
    print(f'Sum of Squared Errors: {summ_sq_r}')
    print(f'Total Sum of Squares: {total_summ_sq}')
    print(f'MSE: {round(mse * 100, 4)}%')
    
    # Estimate sigma_T from residuals
    sigma_T = np.sqrt(np.var(residuals) / delta_t)
    
    return r, sigma, sigma_T

delta_t = 1  # Time step
K = max(data)  # The carrying capacity

# Compute parameters
r, sigma, sigma_T = compute_parameters(data, T, delta_t, K)
print(f'My Estimated Params are: [r: {r}, sigma: {sigma}, sigma_T: {sigma_T}]')

Sum of Squared Errors: 0.07283789238160787
Total Sum of Squares: 0.0745306738613303
MSE: 0.1194%
My Estimated Params are: [r: 6.540839871790122e-05, sigma: 0.10508962980192249, sigma_T: 0.03455517884989635]


# PYTHON IMPLEMENTATION FOR PARAMETER VALUES OF THE CROP YIELD MODEL AFTER INCORPORATING PRECIPITATION

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math
np.random.seed(12)

## THE PREPARATION AND IMPUTATION OF MY PRECIPITATION AND YIELD DATA 

In [5]:
#Precipitation data
rr=pd.read_csv('observed-annual-precipitation-of-ghana-for-1901-2022.csv')
data_inn = np.array(rr['Annual Mean'])
data_precip = data_inn[-62:]
yrs = np.array(rr['Category'])
years = yrs[-62:]

# Log transformation
def log_transform(data_precip):
    # Adding 1 to avoid log(0) if there are zero values
    R = np.log(data_precip + 1)
    return R

R = log_transform(data_precip)


#Crop Yield data
pp=pd.read_csv('crop yield.csv')
data_crop = np.array(pp['Value'])
years = np.array(pp['Year'])

# Log transformation
def log_transform(data_crop):
    # Adding 1 to avoid log(0) if there are zero values
    data = np.log(data_crop + 1)
    return data

data = log_transform(data_crop)

## ESTIMATION OF PARAMETERS FOR THE CROP YIELD MODEL

In [6]:
# Step 1: Computation of Y and X
def compute_Y_X(data, R, delta_t, K):
    n = len(data)
    if n <= 1:
        raise ValueError("Data length must be greater than 1")

    Y = np.zeros(n - 1)
    X = np.zeros((n - 1, 2))
    
    for i in range(n - 1):
        sqrt_R_i = np.sqrt(R[i])
        Y[i] = (data[i + 1] - data[i]) / sqrt_R_i
        X[i, 0] = (1 - np.exp(data[i]) / K) * (delta_t / sqrt_R_i)  
        X[i, 1] = -(delta_t / sqrt_R_i)                            
    
    return Y, X

# Step 2: Computation of C_hat
def compute_C_hat(Y, X, lambda_reg=1e-5):
    n, p = X.shape
    I = np.eye(p)
    X_transpose_X_reg = X.T @ X + lambda_reg * I
    X_transpose_X_inv = np.linalg.inv(X_transpose_X_reg)
    C_hat = X_transpose_X_inv @ X.T @ Y
    return C_hat

# Step 3: Computation of Parameters
def compute_parameters(data, R, delta_t, K, lambda_reg=1e-5):
    Y, X = compute_Y_X(data, R, delta_t, K)
    C_hat = compute_C_hat(Y, X, lambda_reg)
    
    r = C_hat[0]
    sigma = -C_hat[1]
    
    # Predicted Model Y
    Y_pred = X @ C_hat
    residuals = Y - Y_pred
    
    # Calculation of Mean Squared Error (MSE)
    summ_sq_r = np.sum(residuals ** 2)
    total_summ_sq = np.sum((Y - np.mean(Y)) ** 2)
    mse = summ_sq_r / len(Y)
    
    print(f'Sum of Squared Errors: {summ_sq_r}')
    print(f'Total Sum of Squares: {total_summ_sq}')
    print(f'MSE: {round(mse * 100, 4)}%')
    
    # Estimate sigma_T from residuals
    sigma_R = np.sqrt(np.var(residuals) / delta_t)
    
    return r, sigma, sigma_R

delta_t = 1  # Time step
K = max(data)  # The carrying capacity

# Compute parameters
r, sigma, sigma_R = compute_parameters(data, R, delta_t, K)
print(f'My Estimated Params are: [r: {r}, sigma: {sigma}, sigma_R: {sigma_R}]')

Sum of Squared Errors: 0.2861154102074147
Total Sum of Squares: 0.2936348202728435
MSE: 0.469%
My Estimated Params are: [r: 6.905111233245001e-05, sigma: 0.11145631789946434, sigma_R: 0.06848646965721494]
